<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
</center>
<center>Автор материала: программист-исследователь Mail.ru Group, старший преподаватель <br>Факультета Компьютерных Наук ВШЭ Юрий Кашницкий

# <center> Домашнее задание № 8. Часть 2
## <center> Vowpal Wabbit в задаче классификации тегов вопросов на Stackoverflow

## План 2 части домашнего задания
    2.1. Введение
    2.2. Описание данных
    2.3. Предобработка данных
    2.4. Обучение и проверка моделей
    2.5. Заключение

### 2.1. Введение

В этом задании вы будете делать примерно то же, что я каждую неделю –  в Mail.ru Group: обучать модели на выборке в несколько гигабайт. Задание можно выполнить и на Windows с Python, но я рекомендую поработать под \*NIX-системой (например, через Docker) и активно использовать язык bash.
Немного снобизма (простите, но правда): если вы захотите работать в лучших компаниях мира в области ML, вам все равно понадобится опыт работы с bash под UNIX.

[Веб-форма](https://goo.gl/forms/z8zENbMiaEAeB7nG3) для ответов.

Для выполнения задания понадобится установленный Vowpal Wabbit (уже есть в докер-контейнере курса, см. инструкцию в README [репозитория](https://github.com/Yorko/mlcourse_open) нашего курса) и примерно 70 Гб дискового пространства. Я тестировал решение не на каком-то суперкомпе, а на Macbook Pro 2015 (8 ядер, 16 Гб памяти), и самая тяжеловесная модель обучалась около 12 минут, так что задание реально выполнить и с простым железом. Но если вы планируете когда-либо арендовать сервера Amazon, можно попробовать это сделать уже сейчас.

Материалы в помощь:
 - интерактивный [тьюториал](https://www.codecademy.com/en/courses/learn-the-command-line/lessons/environment/exercises/bash-profile) CodeAcademy по утилитам командной строки UNIX (примерно на час-полтора)
 - [статья](https://habrahabr.ru/post/280562/) про то, как арендовать на Amazon машину (еще раз: это не обязательно для выполнения задания, но будет хорошим опытом, если вы это делаете впервые)

### 2.2. Описание данных

Имеются 10 Гб вопросов со StackOverflow – [скачайте](https://yadi.sk/d/krikdUic3Ggjyf) эту выборку. 

Формат данных простой:<br>
<center>*текст вопроса* (слова через пробел) TAB *теги вопроса* (через пробел)

Здесь TAB – это символ табуляции.
Пример первой записи в выборке:

In [2]:
!head -1 stackoverflow.10kk.tsv

 is there a way to apply a background color through css at the tr level i can apply it at the td level like this my td background color e8e8e8 background e8e8e8 however the background color doesn t seem to get applied when i attempt to apply the background color at the tr level like this my tr background color e8e8e8 background e8e8e8 is there a css trick to making this work or does css not natively support this for some reason 	css css3 css-selectors


Здесь у нас текст вопроса, затем табуляция и теги вопроса: *css, css3* и *css-selectors*. Всего в выборке таких вопросов 10 миллионов. 

In [3]:
%%time
!wc -l stackoverflow.10kk.tsv

10000000 stackoverflow.10kk.tsv
CPU times: user 156 ms, sys: 32 ms, total: 188 ms
Wall time: 6.43 s


Обратите внимание на то, что такие данные я уже не хочу загружать в оперативную память и, пока можно, буду пользоваться эффективными утилитами UNIX –  head, tail, wc, cat, cut и прочими.

### 2.3. Предобработка данных

Давайте выберем в наших данных все вопросы с тегами *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift* и подготовим обучающую выборку в формате Vowpal Wabbit. Будем решать задачу 10-классовой классификации вопросов по перечисленным тегам.

Вообще, как мы видим, у каждого вопроса может быть несколько тегов, но мы упростим себе задачу и будем у каждого вопроса выбирать один из перечисленных тегов либо игнорировать вопрос, если таковых тегов нет. 
Но вообще VW поддерживает multilabel classification (аргумент  --multilabel_oaa).
<br>
<br>
Реализуйте в виде отдельного файла `preprocess.py` код для подготовки данных. Он должен отобрать строки, в которых есть перечисленные теги, и переписать их в отдельный файл в формат Vowpal Wabbit. Детали:
 - скрипт должен работать с аргументами командной строки: с путями к файлам на входе и на выходе
 - строки обрабатываются по одной (можно использовать tqdm для подсчета числа итераций)
 - если табуляций в строке нет или их больше одной, считаем строку поврежденной и пропускаем
 - в противном случае смотрим, сколько в строке тегов из списка *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift*. Если ровно один, то записываем строку в выходной файл в формате VW: `label | text`, где `label` – число от 1 до 10 (1 - *javascript*, ... 10 – *swift*). Пропускаем те строки, где интересующих тегов больше или меньше одного 
 - из текста вопроса надо выкинуть двоеточия и вертикальные палки, если они есть – в VW это спецсимволы

In [4]:
import os
from tqdm import tqdm
from time import time
import numpy as np
from sklearn.metrics import accuracy_score

Должно получиться вот такое число строк – 4389054. Как видите, 10 Гб у меня обработались примерно за полторы минуты.

In [12]:
!python preprocess.py stackoverflow.10kk.tsv stackoverflow.vw

10000000it [02:54, 57199.34it/s]
4389054 lines selected, 15 lines corrupted.


Поделите выборку на обучающую, проверочную и тестовую части в равной пропорции - по  1463018 в каждый файл. Перемешивать не надо, первые 1463018 строк должны пойти в обучающую часть `stackoverflow_train.vw`, последние 1463018 – в тестовую `stackoverflow_test.vw`, оставшиеся – в проверочную `stackoverflow_valid.vw`. 

Также сохраните векторы ответов для проверочной и тестовой выборки в отдельные файлы `stackoverflow_valid_labels.txt` и `stackoverflow_test_labels.txt`.

Тут вам помогут утилиты `head`, `tail`, `split`, `cat` и `cut`.

In [13]:
!head -n 1463018 stackoverflow.vw > stackoverflow_train.vw

In [14]:
!tail -n 1463018 stackoverflow.vw > stackoverflow_test.vw

In [15]:
!head -n 2926036 stackoverflow.vw | tail -n 1463018 > stackoverflow_valid.vw

In [17]:
!cut -d '|' -f 1 stackoverflow_valid.vw > stackoverflow_valid_labels.txt

In [18]:
!cut -d '|' -f 1 stackoverflow_test.vw > stackoverflow_test_labels.txt

In [19]:
!head stackoverflow_test_labels.txt

9 
1 
7 
5 
7 
9 
7 
2 
1 
3 


### 2.4. Обучение и проверка моделей

Обучите Vowpal Wabbit на выборке `stackoverflow_train.vw` 9 раз, перебирая параметры passes (1,3,5), ngram (1,2,3).
Остальные параметры укажите следующие: bit_precision=28 и seed=17. Также скажите VW, что это 10-классовая задача.

Проверяйте долю правильных ответов на выборке `stackoverflow_valid.vw`. Выберите лучшую модель и проверьте качество на выборке `stackoverflow_test.vw`.

In [ ]:
# NOTE: При passes > 1 необходимо использовать параметр -c ("cache"), иначе все ломается

In [1]:
!vw --oaa 10 --passes 1 --ngram 1 -b 28 --random_seed 17 -d stackoverflow_train.vw -f stackoverflow_model_1_1.vw --quiet
!vw --oaa 10 --passes 1 --ngram 2 -b 28 --random_seed 17 -d stackoverflow_train.vw -f stackoverflow_model_1_2.vw --quiet
!vw --oaa 10 --passes 1 --ngram 3 -b 28 --random_seed 17 -d stackoverflow_train.vw -f stackoverflow_model_1_3.vw --quiet
!vw --oaa 10 --passes 3 --ngram 1 -b 28 --random_seed 17 -d stackoverflow_train.vw -f stackoverflow_model_3_1.vw --quiet -c
!vw --oaa 10 --passes 3 --ngram 2 -b 28 --random_seed 17 -d stackoverflow_train.vw -f stackoverflow_model_3_2.vw --quiet -c
!vw --oaa 10 --passes 3 --ngram 3 -b 28 --random_seed 17 -d stackoverflow_train.vw -f stackoverflow_model_3_3.vw --quiet -c
!vw --oaa 10 --passes 5 --ngram 1 -b 28 --random_seed 17 -d stackoverflow_train.vw -f stackoverflow_model_5_1.vw --quiet -c
!vw --oaa 10 --passes 5 --ngram 2 -b 28 --random_seed 17 -d stackoverflow_train.vw -f stackoverflow_model_5_2.vw --quiet -c
!vw --oaa 10 --passes 5 --ngram 3 -b 28 --random_seed 17 -d stackoverflow_train.vw -f stackoverflow_model_5_3.vw --quiet -c

In [2]:
!vw -i stackoverflow_model_1_1.vw -t -d stackoverflow_valid.vw -p stackoverflow_model_1_1_valid_predictions.txt --quiet
!vw -i stackoverflow_model_1_2.vw -t -d stackoverflow_valid.vw -p stackoverflow_model_1_2_valid_predictions.txt --quiet
!vw -i stackoverflow_model_1_3.vw -t -d stackoverflow_valid.vw -p stackoverflow_model_1_3_valid_predictions.txt --quiet
!vw -i stackoverflow_model_3_1.vw -t -d stackoverflow_valid.vw -p stackoverflow_model_3_1_valid_predictions.txt --quiet
!vw -i stackoverflow_model_3_2.vw -t -d stackoverflow_valid.vw -p stackoverflow_model_3_2_valid_predictions.txt --quiet
!vw -i stackoverflow_model_3_3.vw -t -d stackoverflow_valid.vw -p stackoverflow_model_3_3_valid_predictions.txt --quiet
!vw -i stackoverflow_model_5_1.vw -t -d stackoverflow_valid.vw -p stackoverflow_model_5_1_valid_predictions.txt --quiet
!vw -i stackoverflow_model_5_2.vw -t -d stackoverflow_valid.vw -p stackoverflow_model_5_2_valid_predictions.txt --quiet
!vw -i stackoverflow_model_5_3.vw -t -d stackoverflow_valid.vw -p stackoverflow_model_5_3_valid_predictions.txt --quiet

Generating 1-grams for all namespaces.
only testing
predictions = stackoverflow_model_1_1_valid_predictions.txt
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = stackoverflow_valid.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        2        2      178
0.000000 0.000000            2            2.0        7        7       74
0.000000 0.000000            4            4.0        5        5      259
0.000000 0.000000            8            8.0        7        7      144
0.000000 0.000000           16           16.0        6        6      359
0.000000 0.000000           32           32.0        2        2      400
0.062500 0.125000           64           64.0        5        5     1061
0.070312 0.078125          128          128.0        2        2      132
0.085938 0.10156

In [8]:
with open('stackoverflow_valid_labels.txt') as pred_file:
    true_labels = [int(label) for label in pred_file.readlines()]

for passes in [1, 3, 5]:
    for ngrams in [1, 2, 3]:
        with open('stackoverflow_model_%d_%d_valid_predictions.txt' % (passes, ngrams)) as pred_file:
            pred_labels = [int(label) for label in pred_file.readlines()]

        print("%d passes, %d-grams: %.3f" % (passes, ngrams, accuracy_score(true_labels, pred_labels)))

1 passes, 1-grams: 0.915
1 passes, 2-grams: 0.931
1 passes, 3-grams: 0.929
3 passes, 1-grams: 0.914
3 passes, 2-grams: 0.928
3 passes, 3-grams: 0.926
5 passes, 1-grams: 0.914
5 passes, 2-grams: 0.929
5 passes, 3-grams: 0.926


<font color='red'> Вопрос 1.</font> Какое сочетание параметров дает наибольшую долю правильных ответов на проверочной выборке `stackoverflow_valid.vw`?
- Биграммы и 3 прохода по выборке
- Триграммы и 5 проходов по выборке
- **Биграммы и 1 проход по выборке**
- Униграммы и 1 проход по выборке

Проверьте лучшую (по доле правильных ответов на валидации) модель на тестовой выборке. 

In [9]:
!vw -i stackoverflow_model_1_2.vw -t -d stackoverflow_test.vw -p stackoverflow_model_1_2_test_predictions.txt

Generating 2-grams for all namespaces.
only testing
predictions = stackoverflow_model_1_2_test_predictions.txt
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = stackoverflow_test.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        9        9      370
0.000000 0.000000            2            2.0        1        1       62
0.000000 0.000000            4            4.0        5        5      158
0.000000 0.000000            8            8.0        2        2      116
0.000000 0.000000           16           16.0        6        6       74
0.031250 0.062500           32           32.0        1        1      324
0.046875 0.062500           64           64.0        4        4       52
0.054688 0.062500          128          128.0        7        7      250
0.066406 0.078125 

In [10]:
passes = 1
ngrams = 2

with open('stackoverflow_test_labels.txt') as pred_file:
    true_labels = [int(label) for label in pred_file.readlines()]

with open('stackoverflow_model_%d_%d_test_predictions.txt' % (passes, ngrams)) as pred_file:
    pred_labels = [int(label) for label in pred_file.readlines()]

print("%d passes, %d-grams: %.3f" % (passes, ngrams, accuracy_score(true_labels, pred_labels)))

1 passes, 2-grams: 0.931


<font color='red'> Вопрос 2.</font> Как соотносятся доли правильных ответов лучшей (по доле правильных ответов на валидации) модели на проверочной и на тестовой выборках? (здесь % – это процентный пункт, т.е., скажем, снижение с 50% до 40% – это на 10%, а не 20%).
- На тестовой ниже примерно на 2%
- На тестовой ниже примерно на 3%
- **Результаты почти одинаковы – отличаются меньше чем на 0.5%**

Обучите VW с параметрами, подобранными на проверочной выборке, теперь на объединении обучающей и проверочной выборок. Посчитайте долю правильных ответов на тестовой выборке. 

In [11]:
! cat stackoverflow_train.vw stackoverflow_valid.vw > stackoverflow_train_plus_valid.vw

In [12]:
!vw --oaa 10 --passes 1 --ngram 2 -b 28 --random_seed 17 -d stackoverflow_train_plus_valid.vw -f stackoverflow_model_b_1_2.vw --quiet -c

In [13]:
!vw -i stackoverflow_model_b_1_2.vw -t -d stackoverflow_test.vw -p stackoverflow_model_b_1_2_test_predictions.txt

Generating 2-grams for all namespaces.
only testing
predictions = stackoverflow_model_b_1_2_test_predictions.txt
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = stackoverflow_test.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        9        9      370
0.000000 0.000000            2            2.0        1        1       62
0.000000 0.000000            4            4.0        5        5      158
0.000000 0.000000            8            8.0        2        2      116
0.000000 0.000000           16           16.0        6        6       74
0.031250 0.062500           32           32.0        1        1      324
0.062500 0.093750           64           64.0        4        2       52
0.062500 0.062500          128          128.0        7        7      250
0.066406 0.07031

In [14]:
passes = 1
ngrams = 2

with open('stackoverflow_test_labels.txt') as pred_file:
    true_labels = [int(label) for label in pred_file.readlines()]

with open('stackoverflow_model_b_%d_%d_test_predictions.txt' % (passes, ngrams)) as pred_file:
    pred_labels = [int(label) for label in pred_file.readlines()]

print("%d passes, %d-grams: %.3f" % (passes, ngrams, accuracy_score(true_labels, pred_labels)))

1 passes, 2-grams: 0.935


<font color='red'> Вопрос 3.</font> На сколько процентных пунктов повысилась доля правильных ответов модели после обучения на вдвое большей выборке (обучающая `stackoverflow_train.vw` + проверочная `stackoverflow_valid.vw`) по сравнению с моделью, обученной только на `stackoverflow_train.vw`?
 - 0.1%
 - **0.4%**
 - 0.8%
 - 1.2%

### 2.5. Заключение

В этом задании мы только познакомились с Vowpal Wabbit. Что еще можно попробовать:
 - Классификация с несколькими ответами (multilabel classification, аргумент  `multilabel_oaa`) – формат данных тут как раз под такую задачу
 - Настройка параметров VW с hyperopt, авторы библиотеки утверждают, что качество должно сильно зависеть от параметров изменения шага градиентного спуска (`initial_t` и `power_t`). Также можно потестировать разные функции потерь – обучать логистическую регресиию или линейный SVM
 - Познакомиться с факторизационными машинами и их реализацией в VW (аргумент `lrq`)